## Prepare data

In [1]:
import os
from collections import defaultdict
import numpy as np
from tqdm.notebook import tqdm

import torch

from yolov3.utils import json_load, json_dump
from yolov3.coco import prepare_data, load_classes
from yolov3.data import read_index
from yolov3.evaluate import evaluate
from yolov3.modules import Darknet
from yolov3.bbox import bbox_iou
from yolov3.metrics import bbox_match, compute_interpolated_precision

In [2]:
coco_val_path = '/media/semyon/Data/Documents/coco/val2017'
annotations_path = '/media/semyon/Data/Documents/coco/annotations/instances_val2017_nocrowd.json'
coco_val_formated_path = '/media/semyon/Data/Documents/coco/val2017_formated'

In [3]:
if not os.path.exists(coco_val_formated_path):
    os.mkdir(coco_val_formated_path)

In [4]:
coco_cat_ids_mapping_path = '/media/semyon/Data/Documents/coco/coco_cat_ids_mapping.json'
coco_cat_ids_mapping = json_load(coco_cat_ids_mapping_path)
coco_cat_ids_mapping = {int(k) - 1: v for k, v in coco_cat_ids_mapping.items()}
coco_cat_ids_mapping_inv = {v: k for k, v in coco_cat_ids_mapping.items()}

In [5]:
prepare_data(coco_val_path, annotations_path, coco_val_formated_path, cat_id_mapping=coco_cat_ids_mapping_inv)

create output dirs
copy images
done       5000/5000
save images index
convert annotations
indexing annotations
done       5000/5000
save annotations index


## Evaluate data

In [5]:
classes = load_classes('config/yolov3/coco.names')
num_classes = len(classes)

In [6]:
model = Darknet('config/yolov3/yolov3_320.cfg')
model.load_weights('config/yolov3/yolov3.weights')

In [7]:
images_path = os.path.join(coco_val_formated_path, 'images')
preds_path = '/media/semyon/Data/Documents/coco/val2017_preds'

In [8]:
if not os.path.exists(preds_path):
    os.mkdir(preds_path)

In [12]:
evaluate(model, images_path, preds_path, num_classes, verbose=True, device='cuda:0', batch_size=4)

read index
evaluation
done       5000/5000
save index


## Compute metrics

In [9]:
new_annotations_path = os.path.join(coco_val_formated_path, 'annotations')
anns_index_path = os.path.join(new_annotations_path, 'index.json')
anns_index = read_index(anns_index_path)
preds_index_path = os.path.join(preds_path, 'index.json')
preds_index = read_index(preds_index_path)

In [10]:
num_preds = 100
iou_thresholds = torch.arange(0.5, 1., 0.05).tolist()
area_ranges = [
    [0 ** 2, 1e5 ** 2],
    [0, 32 ** 2],
    [32 ** 2, 96 ** 2],
    [96 ** 2, 1e5 ** 2]
]
matches_cat = defaultdict(list)
for image_id in tqdm(anns_index):
    target_file = anns_index[image_id]
    target_path = os.path.join(new_annotations_path, target_file)
    targets = torch.load(target_path)
    
    pred_file = preds_index[image_id]
    pred_path = os.path.join(preds_path, pred_file)
    preds = torch.load(pred_path)
    
    if targets.numel() == 0 and preds.numel() == 0:
        continue
    cats = torch.unique(torch.cat((preds[:, 5], targets[:, 4]))).detach().cpu().numpy()
    for c in cats:
        p = preds[preds[:, 5] == c]
        t = targets[targets[:, 4] == c]
        ious = bbox_iou(p[:, :4], t[:, :4])
        target_ids = t[:, 5].detach().cpu().numpy().astype(np.int64)
        match = [bbox_match(
            p[:, :5], t[:, :4], ious,
            iou_thresholds=iou_thresholds,
            num_preds=num_preds,
            area_range=area_range
        ) for area_range in area_ranges]
        matches_cat[c].append(match)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [11]:
num_preds = 100
num_points = 101
num_thresholds = len(iou_thresholds)
num_areas = len(area_ranges)
results = torch.full((num_areas, num_thresholds, num_classes, num_points), -1, dtype=torch.float32)
for a in range(4):
    for c in range(num_classes):
        if c not in matches_cat:
            print('no class', c)
            continue
        m = [v[a] for v in matches_cat[c]]
        num_targets = torch.sum(torch.cat([1 - rec['target_ignore'] for rec in m]))
        num_ignore = torch.sum(torch.cat([rec['target_ignore'] for rec in m]))
        bbox_c = torch.cat([rec['confidences'][:num_preds] for rec in m])
        bbox_m = torch.cat([rec['preds_matched'][:, :num_preds] for rec in m], dim=1)
        bbox_mask = torch.cat([rec['preds_mask'][:, :num_preds] for rec in m], dim=1)
        indices = np.argsort(-bbox_c.detach().cpu().numpy(), kind='mergesort')
        indices = torch.tensor(indices, dtype=torch.int64)
        bbox_c = bbox_c[indices]
        bbox_m = bbox_m[:, indices]
        bbox_mask = bbox_mask[:, indices]
        tp = ((bbox_m != -1) & (bbox_mask == 0)).cumsum(1)
        fp = ((bbox_m == -1) & (bbox_mask == 0)).cumsum(1)
        precision = tp / (tp + fp + 1e-16)
        recall = tp / num_targets
        rthresh = torch.linspace(0, 1, num_points)
        iprecision = torch.empty(num_thresholds, num_points, dtype=torch.float32)
        for i in range(num_thresholds):
            iprecision[i] = compute_interpolated_precision(precision[i], recall[i], rthresh)
        results[a, :, c] = iprecision

In [12]:
# all
r = results[0]
ap = r[r > -1].mean().item()
ap50 = r[0][r[0] > -1].mean().item()
ap75 = r[5][r[5] > -1].mean().item()
print(f'AP(all)={ap:0.3f}')
print(f'AP(all, 0.5)={ap50:0.3f}')
print(f'AP(all, 0.75)={ap75:0.3f}')
# small
r = results[1]
ap = r[r > -1].mean().item()
print(f'AP(small)={ap:0.3f}')
# medium
r = results[2]
ap = r[r > -1].mean().item()
print(f'AP(medium)={ap:0.3f}')
# large
r = results[3]
ap = r[r > -1].mean().item()
print(f'AP(large)={ap:0.3f}')

AP(all)=0.293
AP(all, 0.5)=0.505
AP(all, 0.75)=0.305
AP(small)=0.074
AP(medium)=0.247
AP(large)=0.458


## Bbox areas for COCO annotations

In [34]:
annotations = json_load(annotations_path)
anns = annotations['annotations']
for v in anns:
    w, h = v['bbox'][2:]
    v['area'] = w * h
json_dump(annotations, '/media/semyon/Data/Documents/coco/annotations/instances_val2017_nocrowd_bbox_area.json')